In [1]:
from __future__ import print_function
from __future__ import division

import os
import time
import curses
import numpy as np
import cv2
import pyglet

import src.viztools as vv
import src.utils as util
import src.auto as auto

 ### TODO
 * refactor key input handling across classes (change phase keys in Alien)
 * border zoom out black masking

In [2]:
# ----------------------------------------------------------------------------
# pyglet
# Copyright (c) 2006-2007 Alex Holkner
# Copyright (c) 2007 Andrew Straw
# All rights reserved.
# 
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions 
# are met:
#
#  * Redistributions of source code must retain the above copyright
#    notice, this list of conditions and the following disclaimer.
#  * Redistributions in binary form must reproduce the above copyright 
#    notice, this list of conditions and the following disclaimer in
#    the documentation and/or other materials provided with the
#    distribution.
#  * Neither the name of the pyglet nor the names of its
#    contributors may be used to endorse or promote products
#    derived from this software without specific prior written
#    permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
# "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
# LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS
# FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE
# COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT,
# INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING,
# BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
# LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT
# LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN
# ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
# ----------------------------------------------------------------------------

class NumpyImage(pyglet.image.ImageData):
    def __init__(self,arr,format=None):
        '''Initialize numpy-based image data.

        :Parameters:
            `arr` : array
                numpy array of data. If rank 2, the shape must be
                (height, width). If rank 3, the shape is (depth,
                height, width). Must be C contiguous.
            `format` : str or None
                If specified, a format string describing the numpy
                array ('L' or 'RGB'). Defaults to a format determined
                from the shape of the array.

        '''
        arr = np.asarray(arr)
        if not arr.flags['C_CONTIGUOUS']:
            raise ValueError('numpy array must be C contiguous')
        if len(arr.shape)==2:
            height,width = arr.shape
            if format is None:
                format = 'L'
        elif len(arr.shape)==3:
            height,width,depth = arr.shape
            if format is None:
                if depth==3:
                    format = 'RGB'
                elif depth==4:
                    format = 'RGBA'
                elif depth==1:
                    format = 'L'
                else:
                    raise ValueError("could not determine a format for depth %d"%depth)
        else:
            raise ValueError("array must be rank 2 or rank 3")
        data = None
        pitch = arr.strides[0]
        super(NumpyImage, self).__init__(width, height, format, data, pitch=pitch)
        self.arr = arr
        self.view_new_array(arr)
        
    def _convert(self, format, pitch):
        if format == self._current_format and pitch == self._current_pitch:
            return self.numpy_data_ptr
        else:
            raise NotImplementedError("no support for changing numpy format/pitch")

    def _ensure_string_data(self):
        raise RuntimeError("we should never get here - we are trying to avoid data copying")

    def dirty(self):
        '''Force an update of the texture data.
        '''

        texture = self.texture
        internalformat = None
        self.blit_to_texture( texture.target, texture.level, 0, 0, 0, internalformat )
        
    def view_new_array(self,arr):
        '''View a new numpy array of the same shape.

        The same texture will be kept, but the data from the new array
        will be loaded.

        :Parameters:
            `arr` : array
                numpy array of data. If rank 2, the shape must be
                (height, width). If rank 3, the shape is (depth,
                height, width).
        '''
        arr = np.asarray(arr)
        if arr.shape != self.arr.shape:
            raise ValueError("NumpyImage shape changed!")
        if not arr.dtype == np.uint8:
            raise ValueError("only uint8 numpy arrays supported")
        self.numpy_data_ptr = arr.ctypes.data
        self.arr = arr # maintain a reference to numpy array so it's not de-allocated
        self.dirty()

In [4]:
from pyglet.window import key

disp_full_screen = False
frame_width = int(1980/2)
frame_height = int(1200/2)

# create window with pyglet
if disp_full_screen:
    window = pyglet.window.Window(fullscreen=True)
else:
    window = pyglet.window.Window(
        frame_width, frame_height, 
        resizable=False,
        caption='vid-viz')
    window.set_location(0, 0)

image = pyglet.image.load('/media/data/Dropbox/Git/vid-viz/data/tree-00.jpg')
frame = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/tree-00.jpg')
frame1 = frame[:600,:500,:]

key_list = [False for i in range(256)]
ni = NumpyImage(frame)

@window.event
def on_draw():
    window.clear()
    ni.texture.blit(0, 0)
    
@window.event
def on_key_press(symbol, modifiers):
    if symbol == key.A:
        print('The "A" key was pressed.')
    elif symbol == key.LEFT:
        print('The left arrow key was pressed.')
        ni.view_new_array(frame)
    elif symbol == key.ENTER:
        print('The enter key was pressed.')
        # frame = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/waves-04.jpg')
        ni.view_new_array(np.zeros_like(frame))
    elif symbol == key.ESCAPE:
        key_list[0] = True

while(True):
    
    pyglet.clock.tick()

    for window in pyglet.app.windows:
        window.switch_to()
        window.dispatch_events()
        window.dispatch_event('on_draw')
        window.flip()
    
    if key_list[0]:
        break
        
window.close()

The enter key was pressed.
The left arrow key was pressed.
The enter key was pressed.
The left arrow key was pressed.
The enter key was pressed.
The left arrow key was pressed.
The enter key was pressed.
The left arrow key was pressed.
The enter key was pressed.
The left arrow key was pressed.
The enter key was pressed.
The left arrow key was pressed.
The enter key was pressed.
The left arrow key was pressed.


In [ ]:
whos

In [ ]:
"""vid-viz main loop using pyglet to handle window creation

KEYBOARD INPUTS:
    number keys - choose effect (see viztools.py for detailed keyboard input for each effect)
        0 - thresholding
        1 - alien
        2 - rgbwalk
        3 - rgbburst
        4 - huebloom
        5 - hueswirl
        6 - huecrusher
    q - quit effect (then number keys are once again available for choosing a new effect)
    ` - enable toggling of border effects ('t' to toggle, tab to switch processing/border order)
    spacebar - cycle through sources
    backspace - quit border effect editing
    esc - exit loop
"""

# file_type options : 'cam' | 'video' | 'image' | 'auto'
source_index = 0
source_list = []
source_list.append({
    'file_loc': 'hueswirlchain',
    'file_type': 'auto'})
source_list.append({
    'file_loc': None,
    'file_type': 'cam'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/snowflake-02.mp4',
    'file_type': 'video'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/tree-00.jpg',
    'file_type': 'image'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/honeycomb-00.jpg',
    'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-00.jpg',
#     'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-01.jpg',
#     'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-02.jpg',
#     'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-03.jpg',
#     'file_type': 'image'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-04.jpg',
    'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-05.jpg',
#     'file_type': 'image'})

source = None
num_sources = len(source_list)

# general user parameters
disp_full_screen = False
frame_width = int(1980/2)
frame_height = int(1200/2)
# frame_width = int(1024)
# frame_height = int(768)
target_fps = 20.0

# initialize key handler
effect_index = None
key_list = [False for i in range(256)]

# set pre/post-processing options
post_process = False      # whether or not post-processing is active effect
post_process_pre = False  # whether post-processing pre/proceeds other effects

# initialize processing objects
effects = [
    vv.Threshold(),      # 0
    vv.Alien(),          # 1
    vv.RGBWalk(),        # 2
    vv.RGBBurst(),       # 3
    vv.HueBloom(),       # 4
    vv.HueSwirl(),       # 5
    vv.HueSwirlMover(),  # 6
    vv.HueCrusher()]     # 7
num_effects = len(effects)
border = vv.Border()
postproc = vv.PostProcess()
    
# create window with pyglet
window = pyglet.window.Window()

new_source = True
while(True):

    time_pre = time.time()
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    key_list[key] = True
    if key == 27:
        # escape key; exit program
        break
    elif key == ord('q'):
        # quit current effect
        effect_index = None
    elif key == ord('\b'):
        post_process = False
    elif key == ord('`'):
        post_process = True
    elif key == ord(' '):
        source_index = (source_index + 1) % num_sources
        new_source = True
    elif key == ord('\t'):
        # only change post-processing order if in post-process mode
        if post_process:
            post_process_pre = not post_process_pre
            
    if new_source:
        
        # reset necessary parameters
        new_source = False
        effect_index = None
        fr_count = 0
        for _, effect in enumerate(effects):
            effect.reset()
        border.reset()
        postproc.reset()
        
        # free previous resources
        if source is 'cam' or source is 'video':
            cap.release()
        
        # load source
        source = source_list[source_index]['file_type']
        if source is 'cam':
            cap = cv2.VideoCapture(0)
            TOTAL_FRAME_COUNT = float('inf')
            frame_mask = None
        elif source is 'video':
            cap = cv2.VideoCapture(source_list[source_index]['file_loc'])
            # target_fps = cap.get(cv2.CAP_PROP_FPS)
            TOTAL_FRAME_COUNT = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_mask = None
        elif source is 'image':
            frame_orig = cv2.imread(source_list[source_index]['file_loc'])
            TOTAL_FRAME_COUNT = float('inf')
            frame_mask = np.copy(frame_orig)   
        elif source is 'auto':
            if source_list[source_index]['file_loc'] is 'hueswirlchain':
                auto_effect = auto.HueSwirlChain(frame_width, frame_height)
            else:
                print('Invalid auto effect')
            TOTAL_FRAME_COUNT = float('inf')
        else:
            print('Invalid source type')          
    
    # get frame and relevant info
    if source is 'cam' or source is 'video':
        ret, frame = cap.read()
    elif source is 'image':
        frame = np.copy(frame_orig)
    elif source is 'auto':
        frame = auto_effect.process(key_list)
    
    if source is not 'auto':
        # get uniform frame sizes 
        frame = util.resize(frame, frame_width, frame_height)
    
    # update current effect
    if effect_index is None:
        for num in range(10):
            if key == ord(str(num)):
                effect_index = num
                key_list[key] = False
    
    # apply borders before effect
    if post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)
    
    # process frame
    if source is not 'auto' and effect_index is not None:
        if post_process:
            frame = effects[effect_index].process(frame, key_list, key_lock=True)            
        else:
            frame = effects[effect_index].process(frame, key_list)
            
    # apply borders after effect
    if not post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
#             frame = postproc.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)
#             frame = postproc.process(frame, key_list, key_lock=True)
    
    # display frame

    
    # control animation
    fr_count += 1
    if fr_count == TOTAL_FRAME_COUNT:
        # reset frame postion to 1 (not zero so window isn't moved)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
        fr_count = 1

    # calculate, limit and output fps
    time_tot = time.time() - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_tot = time.time() - time_pre
    print('\r%03i fps' % (1.0 / time_tot), end='')
#     print('\r%02i' % effects[5].PROPS[4]['VAL'], end='')
#     stdscr.addstr(0, 0, '%03i fps' % (1.0 / time_tot))
#     stdscr.addstr(1, 0, '%05i frames' % fr_count)
#     stdscr.refresh()
    
#     frame_final = np.copy(frame)
    
    # reset key list (for pressed keys that are not reset by effect object(s))
    key_list[key] = False
    
if source is 'cam' or source is 'video':
    cap.release()
cv2.destroyWindow('frame')

In [ ]:
"""vid-viz main loop using OpenCV to handle window creation

KEYBOARD INPUTS:
    number keys - choose effect (see viztools.py for detailed keyboard input for each effect)
        0 - thresholding
        1 - alien
        2 - rgbwalk
        3 - rgbburst
        4 - huebloom
        5 - hueswirl
        6 - huecrusher
    q - quit effect (then number keys are once again available for choosing a new effect)
    ` - enable toggling of border effects ('t' to toggle, tab to switch processing/border order)
    spacebar - cycle through sources
    backspace - quit border effect editing
    esc - exit loop
"""

# file_type options : 'cam' | 'video' | 'image' | 'auto'
source_index = 0
source_list = []
source_list.append({
    'file_loc': 'hueswirlchain',
    'file_type': 'auto'})
source_list.append({
    'file_loc': None,
    'file_type': 'cam'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/snowflake-02.mp4',
    'file_type': 'video'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/tree-00.jpg',
    'file_type': 'image'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/honeycomb-00.jpg',
    'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-00.jpg',
#     'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-01.jpg',
#     'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-02.jpg',
#     'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-03.jpg',
#     'file_type': 'image'})
source_list.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-04.jpg',
    'file_type': 'image'})
# source_list.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-05.jpg',
#     'file_type': 'image'})

source = None
num_sources = len(source_list)

# general user parameters
disp_full_screen = False
frame_width = int(1980/2)
frame_height = int(1200/2)
# frame_width = int(1024)
# frame_height = int(768)
target_fps = 20.0

# initialize key handler
effect_index = None
key_list = [False for i in range(256)]

# set pre/post-processing options
post_process = False      # whether or not post-processing is active effect
post_process_pre = False  # whether post-processing pre/proceeds other effects

# initialize processing objects
effects = [
    vv.Threshold(),      # 0
    vv.Alien(),          # 1
    vv.RGBWalk(),        # 2
    vv.RGBBurst(),       # 3
    vv.HueBloom(),       # 4
    vv.HueSwirl(),       # 5
    vv.HueSwirlMover(),  # 6
    vv.HueCrusher()]     # 7
num_effects = len(effects)
border = vv.Border()
postproc = vv.PostProcess()
    
new_source = True
while(True):

    time_pre = time.time()
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    key_list[key] = True
    if key == 27:
        # escape key; exit program
        break
    elif key == ord('q'):
        # quit current effect
        effect_index = None
    elif key == ord('\b'):
        post_process = False
    elif key == ord('`'):
        post_process = True
    elif key == ord(' '):
        source_index = (source_index + 1) % num_sources
        new_source = True
    elif key == ord('\t'):
        # only change post-processing order if in post-process mode
        if post_process:
            post_process_pre = not post_process_pre
            
    if new_source:
        
        # reset necessary parameters
        new_source = False
        effect_index = None
        fr_count = 0
        for _, effect in enumerate(effects):
            effect.reset()
        border.reset()
        postproc.reset()
        
        # free previous resources
        if source is 'cam' or source is 'video':
            cap.release()
        
        # load source
        source = source_list[source_index]['file_type']
        if source is 'cam':
            cap = cv2.VideoCapture(0)
            TOTAL_FRAME_COUNT = float('inf')
            frame_mask = None
        elif source is 'video':
            cap = cv2.VideoCapture(source_list[source_index]['file_loc'])
            # target_fps = cap.get(cv2.CAP_PROP_FPS)
            TOTAL_FRAME_COUNT = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_mask = None
        elif source is 'image':
            frame_orig = cv2.imread(source_list[source_index]['file_loc'])
            TOTAL_FRAME_COUNT = float('inf')
            frame_mask = np.copy(frame_orig)   
        elif source is 'auto':
            if source_list[source_index]['file_loc'] is 'hueswirlchain':
                auto_effect = auto.HueSwirlChain(frame_width, frame_height)
            else:
                print('Invalid auto effect')
            TOTAL_FRAME_COUNT = float('inf')
        else:
            print('Invalid source type')          
    
    # get frame and relevant info
    if source is 'cam' or source is 'video':
        ret, frame = cap.read()
    elif source is 'image':
        frame = np.copy(frame_orig)
    elif source is 'auto':
        frame = auto_effect.process(key_list)
    
    if source is not 'auto':
        # get uniform frame sizes 
        frame = util.resize(frame, frame_width, frame_height)
    
    # update current effect
    if effect_index is None:
        for num in range(10):
            if key == ord(str(num)):
                effect_index = num
                key_list[key] = False
    
    # apply borders before effect
    if post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)
    
    # process frame
    if source is not 'auto' and effect_index is not None:
        if post_process:
            frame = effects[effect_index].process(frame, key_list, key_lock=True)            
        else:
            frame = effects[effect_index].process(frame, key_list)
            
    # apply borders after effect
    if not post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
#             frame = postproc.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)
#             frame = postproc.process(frame, key_list, key_lock=True)
    
    # display frame
    if disp_full_screen:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    # control animation
    fr_count += 1
    if fr_count == TOTAL_FRAME_COUNT:
        # reset frame postion to 1 (not zero so window isn't moved)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
        fr_count = 1

    # calculate, limit and output fps
    time_tot = time.time() - time_pre
    if time_tot < 1/target_fps:
        time.sleep(1/target_fps - time_tot)
    time_tot = time.time() - time_pre
    print('\r%03i fps' % (1.0 / time_tot), end='')
#     print('\r%02i' % effects[5].PROPS[4]['VAL'], end='')
#     stdscr.addstr(0, 0, '%03i fps' % (1.0 / time_tot))
#     stdscr.addstr(1, 0, '%05i frames' % fr_count)
#     stdscr.refresh()
    
#     frame_final = np.copy(frame)
    
    # reset key list (for pressed keys that are not reset by effect object(s))
    key_list[key] = False
    
if source is 'cam' or source is 'video':
    cap.release()
cv2.destroyWindow('frame')

In [ ]:
reload(vv)
reload(auto)
reload(util)

In [ ]:
string = source_list[source_index]['file_loc']
type(string)
type('hueswirl-chain')

In [ ]:
%timeit cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

In [ ]:
import PIL.Image
PIL.Image.fromarray(frame_final).save('/media/data/Dropbox/image_play/hue-bloom/final/tree-00.jpg', 'jpeg')

In [ ]:
img1 = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')
img1 = util.resize(img1, 1920, 1200)

# img0 = 0
# img2 = cv2.addWeighted(img0, 0.1, img1, 0.9, 0)
# %timeit cv2.addWeighted(img0, 0.1, img1, 0.9, 0)

# im_width, im_height, _ = img1.shape
# %timeit cv2.warpAffine(img1, \
#             np.float32([[1, 0, 100], \
#                         [0, 1, 200]]), \
#             (im_width, im_height))

%timeit cv2.threshold(img1, 128, 255, cv2.THRESH_TOZERO)

In [ ]:
frame_back_0 = np.random.rand(2,2,3)
frame_back_1 = np.random.rand(4,4,3)
frame_back0 = cv2.resize(
    frame_back_0,
    (512, 512),
    interpolation=cv2.INTER_LINEAR)
frame_back1 = cv2.resize(
    frame_back_1,
    (512, 512),
    interpolation=cv2.INTER_LINEAR)
frame_back = 255.0 * (frame_back0 / 2 + frame_back1 / 2)
frame_back = frame_back.astype('uint8')


img_noise = np.random.uniform(size=(1024,1024,3))

hw = img_noise.shape[:2]
for i in range(8):
    tmp = np.random.uniform(size=(hw[0]//(2**(i+1)), hw[1]//(2**(i+1)), 3))
#     tmp = np.random.rand(2**(i+1), 2**(i+1), 3)
    img_noise = img_noise + cv2.resize(tmp, hw, interpolation=cv2.INTER_LINEAR)

    
img_noise = 255.0 * (img_noise - np.min(img_noise)) / (np.max(img_noise) - np.min(img_noise))
img_noise = img_noise.astype('uint8')

while True:

    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        # escape key
        break
        
    cv2.imshow('frame', img_noise)
        
cv2.destroyWindow('frame')

In [ ]:
frame = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')
frame = util.resize(frame, int(1920/2), int(1200/2))

# extract hue values
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
[im_height, im_width, _] = frame.shape
hue_final = np.zeros((im_height, im_width), dtype='uint8')

chunk_center = 128
chunk_widths_og = 128
chunk_width = 8
center_offset = 0

# human-readable names
num_chunks = 3
center_offset = 0
chunk_width = 8

chunk_widths_og = int(256 / num_chunks)
chunk_range_mins = np.zeros((num_chunks, 1), 'uint8')
chunk_range_maxs = np.zeros((num_chunks, 1), 'uint8')
chunk_centers = np.zeros((num_chunks, 1), 'uint8')
for chunk in range(num_chunks):
    chunk_range_mins[chunk] = chunk * chunk_widths_og
    chunk_range_maxs[chunk] = (chunk + 1) * chunk_widths_og - 1
    chunk_centers[chunk] = chunk_range_mins[chunk] + int(chunk_widths_og / 2)
chunk_range_maxs[-1] = 255
    
# process image
if len(frame.shape) == 3:
    [im_height, im_width, _] = frame.shape
elif len(frame.shape) == 2:
    [im_height, im_width] = frame.shape

# extract hue values
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
hue_final = np.zeros((im_height, im_width), dtype='uint8')

# scale whole image
scale = chunk_width / chunk_widths_og
frame_scaled = cv2.addWeighted(0, 1 - scale, frame, scale, 0)

for chunk in range(num_chunks):

    chunk_center = chunk_centers[chunk]

    # get rid of values outside bounds of chunk
    hue_mask = cv2.inRange(frame[:, :, 0],
                           chunk_range_mins[chunk],
                           chunk_range_maxs[chunk])
    # hue_mask is now a mask for the location of values in a specific
    # hue band;
    hue = frame_scaled[:, :, 0] + chunk_center - int(chunk_width / 2) + \
        center_offset
    hue = cv2.bitwise_and(hue, hue_mask)

    # put into final hue array (bitwise or takes all nonzero vals)
    hue_final = cv2.bitwise_or(hue_final, hue)

# return to bgr format
frame[:, :, 0] = hue_final
frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)

while(True):
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    print('\r%g' % key, end='')
    
    if key == 27:
        # escape key
        break
        
    cv2.imshow('frame', frame)
        
cv2.destroyWindow('frame')

In [ ]:
img_orig = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/honeycomb_01.jpg')
img_orig = cv2.resize(
    img_orig,
    None,
    fx=0.5,
    fy=0.5,
    interpolation=cv2.INTER_LINEAR)

# get mask
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
img_gray = cv2.medianBlur(img_gray, 11)
img_mask = cv2.adaptiveThreshold(
    img_gray,
    255, # thresh ceil
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY_INV,
    21, # thresh block
    4 # thresh bias
)
img_mask2 = cv2.adaptiveThreshold(
    img_gray,
    255, # thresh ceil
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY,
    21, # thresh block
    4 # thresh bias
)
[mask_height, mask_width] = img_mask.shape

# get hue background
back_height = 10 #int(mask_height/60.0)
back_width = 10 #int(mask_width/60.0)
img_back = np.ones((back_height, back_width, 3))
img_back[:, :, 0] = np.random.rand(back_height, back_width)
img_back = cv2.resize(img_back,
    (mask_width, mask_height),
    interpolation=cv2.INTER_CUBIC)
img_back = 255.0 * img_back
img_back = img_back.astype('uint8')
img_back = cv2.cvtColor(img_back, cv2.COLOR_HSV2BGR)

# get blurred background
img_back2 = img_back
for chan in range(3):
    img_back2[:, :, chan] = cv2.bitwise_and(img_back[:, :, chan], img_mask)
REDUCE_SIZE = False
if REDUCE_SIZE:
    img_back2 = cv2.resize(
        img_back2,
        None,
        fx=0.10,
        fy=0.10,
        interpolation=cv2.INTER_LINEAR)
    img_back2 = cv2.GaussianBlur(img_back2, (5,5), 0)
    img_back2 = cv2.resize(
        img_back2,
        (mask_width, mask_height),
        interpolation=cv2.INTER_LINEAR)
else:
    img_back2 = cv2.GaussianBlur(img_back2, (71, 71), 0)

# remask blurred background
img = img_back2
for chan in range(3):
    img[:, :, chan] = cv2.bitwise_and(img_back2[:, :, chan], img_mask2)
    
while(True):
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        # escape key
        break
        
    cv2.imshow('frame', img_back2)
    cv2.moveWindow('frame', 0, 0)

cv2.destroyWindow('frame')

In [ ]:
%whos

### To patrol-cycle an mp4 with ffmpeg
* ffmpeg -i source.mp4 -vf reverse source_rev.mp4
* ffmpeg -f concat -i concat_list.txt -c copy concated.mp4

##### concat_list.txt
file '/path/to/vid0.mp4'

file '/path/to/vid1.mp4'